In [29]:
using Pkg
using BSON: @load, @save
using JSON
using DataStructures: DefaultOrderedDict, OrderedDict
using Flux
using NPZ
using FluxGoodies

In [30]:
model_dir = "../exp/full/8/group_data_size/1024_512_256";
transform_json = "../data/full/8/group_data_size.transform.json";

Load model from BSON file.

In [19]:
@load joinpath(model_dir, "model.bson") cpumodel
cpumodel

MLP(Chain(Dense(51, 1024, relu), Dense(1024, 512, relu), Dense(512, 256, relu), Dense(256, 1)))

Generate "random" input data (single sample ones) and evaluate the model for it as a sanity check.

In [25]:
X = ones(51,1)
cpumodel(X)

1×1 Array{Float32,2}:
 9.700974

Export parameters (weights and biases) of all layers.

In [26]:
function export_model(model_dir)
    @load joinpath(model_dir, "model.bson") cpumodel
    ps = collect(params(cpumodel))
    d = Dict{String,Any}()
    nlayers = length(ps) ÷ 2
    for i in 1:nlayers
        d["W$i"] = ps[2i-1]' # Note Python uses row-first approach to store matrices, so every matrix is transposed
        d["b$i"] = ps[2i]'
    end
    npzwrite(joinpath(model_dir, "model.npz"), d)
end

model = export_model(model_dir)

Each input gets standardized before feeding the MLP, hence we need to extract all $\mu$ and $\sigma$ values, too. Let's read them from dataset transformation JSON. We must read them in order, hence the `OrderedDict`.

In [51]:
function export_standardization(model_dir, transform_json)
    transforms = JSON.parsefile(transform_json, dicttype=OrderedDict)["trns"]["X"];
    μs = [trn["μ"] for trn in values(transforms) if trn["type"] == "Standardize"]
    σs = [trn["σ"] for trn in values(transforms) if trn["type"] == "Standardize"]
    d = Dict("mus" => μs, "sigmas" => σs)
    npzwrite(joinpath(model_dir, "model_standardization.npz"), d)
end

export_standardization(model_dir, transform_json)